In [1]:
import os

In [2]:
%pwd

'/home/jafarid/code/yogaposes/research'

In [3]:
os.chdir('../')
%pwd

'/home/jafarid/code/yogaposes'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [5]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    resnet_trained_model_path: Path
    resnet_updated_base_model_path: Path
    traning_data: Path
    params_augmentation: bool
    params_image_size: list 
    params_batch_size: int 
    params_epoches: int
    params_learning_rate: float
    params_classes: int
    params_type: str
    all_params: dict
    mlflow_uri: str

In [6]:
from yogaposes.constants import *
from yogaposes.utils.common import read_yaml, create_directories
from dotenv import load_dotenv
load_dotenv()

MLFLOW_TRACKING_URI = os.environ['MLFLOW_TRACKING_URI']
MLFLOW_TRACKING_USERNAME = os.environ['MLFLOW_TRACKING_USERNAME']
MLFLOW_TRACKING_PASSWORD = os.environ['MLFLOW_TRACKING_PASSWORD']

In [7]:
class configurationManager:
    def __init__(self,config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
        
    def get_traning_config(self) -> TrainingConfig:
        model_training = self.config.model_training
        prepare_base_model = self.config.prepare_base_model
        training_data = os.path.join(self.config.data_ingestion.root_dir, 'yoga-poses-dataset')
        
        create_directories([model_training.root_dir])
        
        training_config = TrainingConfig(root_dir= model_training.resnet_trained_model_path, 
                                        resnet_trained_model_path= model_training.resnet_trained_model_path,
                                        resnet_updated_base_model_path= prepare_base_model.resnet_updated_base_model_path,
                                        traning_data = training_data,
                                        params_augmentation = self.params.AUGMENTATION,
                                        params_image_size = self.params.IMAGE_SIZE,
                                        params_classes = self.params.CLASSES,
                                        params_batch_size= self.params.BATCH_SIZE,
                                        params_epoches = self.params.EPOCHS,
                                        params_learning_rate = self.params.LEARNING_RATE,
                                        params_type = self.params.TYPE,
                                        all_params = self.params,
                                        mlflow_uri= MLFLOW_TRACKING_URI
                                        )
        
        return training_config

In [25]:
import torch
import torch.optim as optim
from torch import nn
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import mlflow
from urllib.parse import urlparse
from PIL import ImageFile, Image
from transformers import ViTFeatureExtractor, ViTForImageClassification, ViTImageProcessor
from transformers import Trainer, TrainingArguments
from torchinfo import summary
import requests
# Evaluation
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [26]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,              # Total number of training epochs
    per_device_train_batch_size=8,   # Batch size per device during training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    warmup_steps=500,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=10,
)

In [31]:
class TransformerModelTrainer(object):
    def __init__(self, config:TrainingConfig):
        self.config = config
        
        self.model, self.feature_extractor, self.processor = self.load_model()
        
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        
        self.train_loader, self.val_loader = self.set_loaders()
        
        
        
    def load_model(self):
        idTolabels = {0:'downdog', 1: 'godess', 2:'plank', 3:'tree', 4:'warrior2'}
        feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
        processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
        model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=self.config.params_classes,ignore_mismatched_sizes=True,id2label=idTolabels)
        
        return model, feature_extractor , processor
    
    def set_seed(self, seed=42):
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    def set_loaders(self):
        
        # Allow loading of truncated images
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        
        # image net statistics
        normalizer = Normalize(mean=self.feature_extractor.image_mean, std=self.feature_extractor.image_std)
        
        composer = Compose([Resize((224,224)), ToTensor(), normalizer])
        
        train_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TRAIN'), transform=composer)
        val_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TEST'), transform=composer)
        
        train_loader = DataLoader(train_data, batch_size=self.config.params_batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=self.config.params_batch_size)
        
        return train_loader, val_loader
    
    def collate_fn(self, examples):
        pixels = torch.stack([example["pixels"] for example in examples])
        labels = torch.tensor([example["label"] for example in examples])
        return {"pixel_values": pixels, "labels": labels}

    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return dict(accuracy=accuracy_score(predictions, labels))
    
    def fine_tune(self, training_args):
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        
        # image net statistics
        normalizer = Normalize(mean=self.feature_extractor.image_mean, std=self.feature_extractor.image_std)
        
        composer = Compose([Resize((224,224)), ToTensor(), normalizer])
        train_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TRAIN'), transform=composer)
        val_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TEST'), transform=composer)
        trainer = Trainer(model=self.model,
                            args=training_args,
                            train_dataset=train_data,
                            eval_dataset=val_data,
                            #tokenizer=self.feature_extractor,
                            data_collator = self.set_loaders,
                            tokenizer = self.processor,
                        )
        trainer.train()
        

    def predict(self,x_filename):
        
        # Allow loading of truncated images
        ImageFile.LOAD_TRUNCATED_IMAGES = True       
        # Load an image
        image = Image.open(x_filename)
        encoding = self.feature_extractor(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**encoding)

        # Extract the predicted class ID
        predicted_class_id = outputs.logits.argmax(-1).item()
       
        
        labels = {0:'downdog', 1: 'godess', 2:'plank', 3:'tree', 4:'warrior2'}
    
        
        return labels[predicted_class_id]
        

        

In [32]:
try:
    config = configurationManager()
    training_config = config.get_traning_config()
    training = TransformerModelTrainer(config=training_config)
    training.fine_tune(training_args)
    c = training.predict('artifacts/data_ingestion/yoga-poses-dataset/DATASET/TRAIN/plank/00000128.jpg')
    print(c)

except Exception as e:
    raise e

[2024-03-14 20:09:09,950: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-14 20:09:09,952: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-14 20:09:09,953: INFO: common: created directory at: artifacts]
[2024-03-14 20:09:09,953: INFO: common: created directory at: artifacts/training]


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/408 [00:00<?, ?it/s]

TypeError: set_loaders() takes 1 positional argument but 2 were given

In [13]:
class ModelTrainer(object):
    def __init__(self, config:TrainingConfig, loss_fn=None, optimizer=None):
        self.config = config
        self.model = self.load_model()
        self.loss_fn = loss_fn if loss_fn else nn.CrossEntropyLoss(reduction='mean')
        self.optimizer = optimizer if optimizer else optim.Adam(self.model.parameters(), lr=self.config.params_learning_rate)
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(self.device)
        
        self.train_loader, self.val_loader = self.set_loaders()
        
        self.losses = []
        self.val_losses = []
        self.accuracy = []
        self.val_accuracy = []
        self.total_epoches = 0
        
        self.train_step_fn = self._make_train_step_fn()
        self.val_step_fn = self._make_val_step_fn()
        
    def load_model(self):
     
        return torch.load(self.config.resnet_updated_base_model_path)
    
    def set_seed(self, seed=42):
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        torch.manual_seed(seed)
        np.random.seed(seed)
    
    def set_loaders(self):
        
        # Allow loading of truncated images
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        
        # image net statistics
        normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
        
        composer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])
        
        train_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TRAIN'), transform=composer)
        val_data = ImageFolder(root=os.path.join(self.config.traning_data,'DATASET/TEST'), transform=composer)
        
        train_loader = DataLoader(train_data, batch_size=self.config.params_batch_size, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=self.config.params_batch_size)
        
        return train_loader, val_loader
    
    # higher order function to be set and built globally and constructed the inner fuction without knowning x and y before hand
    def _make_train_step_fn(self):
        # single batch operation
        def perform_train_step_fn(x,y):
            # set the train mode
            self.model.train()
            
            # step 1: compute model output
            yhat = self.model(x)
            #yhat = yhat.float()
            #y = y.float()
            # step 2: compute the loss
              
            loss= self.loss_fn(yhat,y)
            
            # step 2': compute accuracy 
            yhat = torch.argmax(yhat,1)
            
            total_correct = (yhat ==y).sum().item()
            total = y.shape[0]
            acc = total_correct/total
            
            # step 3: compute the gradient
            loss.backward()
            
            #step4: update parameters
            self.optimizer.step()
            self.optimizer.zero_grad()
            
            #step 5: return the loss
            return loss.item() , acc
        return perform_train_step_fn
    
    def _make_val_step_fn(self):
        # single batch operation
        def perform_val_step_fn(x,y):
            # set the model in val mode
            self.model.eval()
            
            #step 1: compute the prediction
            yhat = self.model(x)
            #yhat = yhat.float()
            #y = y.float()
            #step 2: compute the loss
            loss = self.loss_fn(yhat,y)
            # step 2': compute accuracy 
            yhat = torch.argmax(yhat,1)
          
            total_correct = (yhat ==y).sum().item()
            total = y.shape[0]
            acc = total_correct/total
            
            return loss.item(), acc
        return perform_val_step_fn
    
    def _mini_batch(self, validation=False):
        # one epoch operation 
        if validation:
            data_loader = self.val_loader
            step_fn = self.val_step_fn
            
        else: 
            data_loader = self.train_loader
            step_fn = self.train_step_fn
            
        if data_loader is None:
            return None
        
        mini_batch_losses = []
        mini_batch_accs = []
        for x_batch, y_batch in data_loader:
            x_batch = x_batch.to(self.device)
            y_batch = y_batch.to(self.device)
            
            mini_batch_loss, mini_batch_acc = step_fn(x_batch,y_batch)
            
            mini_batch_losses.append(mini_batch_loss)
            mini_batch_accs.append(mini_batch_acc)
        
        loss = np.mean(mini_batch_losses)
        acc = np.mean(mini_batch_accs)
        return loss, acc
    
    def train(self, seed=42):
        self.set_seed(seed)
        
        for epoch in range(self.config.params_epoches):
            self.total_epoches +=1
            
            # perform training on mini batches within 1 epoch
            loss, acc = self._mini_batch(validation=False)
            self.losses.append(loss)
            self.accuracy.append(acc)
            # now calc validation
            with torch.no_grad():
                val_loss, val_acc = self._mini_batch(validation=True)
                self.val_losses.append(val_loss)
                self.val_accuracy.append(val_acc)
        
            print(f'\nEpoch: {epoch+1} \tTraining Loss: {loss:.4f} \tValidation Loss: {val_loss:.4f}')
            print(f'\t\tTraining Accuracy: {100 * acc:.2f}%\t Validation Accuracy: {100 * val_acc:.2f}%')
                
        self.save_checkpoint()
            
    def save_checkpoint(self):
        checkpoint = {'epoch': self.total_epoches,
                      'model_state_dict': self.model.state_dict(),
                      'optimizer_state_dict': self.optimizer.state_dict(),
                      'loss': self.losses,
                      'accuracy': self.accuracy,
                      'val_loss': self.val_losses,
                      'val_accuracy': self.val_accuracy
                      }
        torch.save(checkpoint, self.config.resnet_trained_model_path)
        
    def load_checkpoint(self):
        checkpoint = torch.load(self.config.resnet_trained_model_path)
        self.model.load_state_dict(checkpoint["model_state_dict"])
        self.optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        self.total_epoches = checkpoint["epoch"]
        self.losses = checkpoint["loss"]
        self.accuracy = checkpoint['accuracy']
        self.val_accuracy = checkpoint['val_accuracy']
        self.val_losses = checkpoint["val_loss"]
        self.model.train() # always use train for resuming traning
        
    def _preprocess_image(self, filename):
        image = Image.open(filename)
        # Allow loading of truncated images
        ImageFile.LOAD_TRUNCATED_IMAGES = True
        
        # image net statistics
        normalizer = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224,0.225])
        
        composer = Compose([Resize(256), CenterCrop(224), ToTensor(), normalizer])
        image = composer(image).unsqueeze(0)
        return image
        
    
    def predict(self,x_filename):
        
        self.load_checkpoint()
        self.model.eval()
        x = self._preprocess_image(x_filename)
        x_tensor = torch.as_tensor(x).float()
        y_hat_tensor = self.model(x_tensor.to(self.device))
        
        # set it back to the train mode
        self.model.train()
        
        labels = {0:'downdog', 1: 'godess', 2:'plank', 3:'tree', 4:'warrior2'}
        prediction=np.argmax(y_hat_tensor.detach().cpu().numpy())
        
        return labels[prediction]
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({'train_loss': np.mean(self.losses),'val_loss': np.mean(self.val_losses), 'train_accuracy': np.mean(self.accuracy), 'val_accuracy': np.mean(self.val_accuracy)})
        
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.pytorch.log_model(self.model, "model", registered_model_name="ResNet18Model")
            else:
                mlflow.pytorch.log_model(self.model, "model")
        
            

In [49]:
try:
    config = configurationManager()
    training_config = config.get_traning_config()
    training = ModelTrainer(config=training_config)
    training.train()
    training.log_into_mlflow()

except Exception as e:
    raise e   

[2024-03-05 02:03:18,404: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-05 02:03:18,406: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 02:03:18,407: INFO: common: created directory at: artifacts]
[2024-03-05 02:03:18,407: INFO: common: created directory at: artifacts/training]


/home/jafarid/miniconda3/envs/yogaposes/lib/python3.8/site-packages/PIL/Image.py:981: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Epoch: 1 	Training Loss: 8.5584 	Validation Loss: 4.1259
		Training Accuracy: 53.55%	 Validation Accuracy: 79.24%

Epoch: 2 	Training Loss: 3.5759 	Validation Loss: 1.1918
		Training Accuracy: 70.33%	 Validation Accuracy: 89.17%

Epoch: 3 	Training Loss: 2.8397 	Validation Loss: 4.4006
		Training Accuracy: 76.90%	 Validation Accuracy: 75.00%

Epoch: 4 	Training Loss: 2.3950 	Validation Loss: 1.7030
		Training Accuracy: 79.78%	 Validation Accuracy: 88.12%

Epoch: 5 	Training Loss: 2.6101 	Validation Loss: 1.0900
		Training Accuracy: 79.40%	 Validation Accuracy: 90.97%


Registered model 'ResNet18Model' already exists. Creating a new version of this model...
2024/03/05 02:06:05 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ResNet18Model, version 3
Created version '3' of model 'ResNet18Model'.


In [ ]:
try:
    config = configurationManager()
    training_config = config.get_traning_config()
    training = ModelTrainer(config=training_config, inference=True)
    training.train()
    training.log_into_mlflow()

except Exception as e:
    raise e   

In [15]:
try:
    config_inference = configurationManager()
    inference_config = config_inference.get_traning_config()
    inference = ModelTrainer(config=inference_config)
    c = inference.predict('artifacts/data_ingestion/yoga-poses-dataset/DATASET/TRAIN/plank/00000128.jpg')
    print(c)

except Exception as e:
    raise e   

[2024-03-05 19:20:36,074: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-05 19:20:36,076: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 19:20:36,077: INFO: common: created directory at: artifacts]
[2024-03-05 19:20:36,077: INFO: common: created directory at: artifacts/training]
plank
